소방청
* 전국소방기관 검색:서울 왼쪽 소방리스트 페이지 넘어가며 100건 리스트 저장
* 관서명, 주소, 전화번호 -> csv로 저장
* kakao api로 위경도 값 삽입, (없는 주소는 drop)
* 지도에 표시
* 페이지를 바꾸는것은 셀레늄으로, bs로 크롤링

In [2]:
import numpy as np
import pandas as pd
import time
import re 

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [11]:
# 메인 홈페이지
url='https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5382842&myY=126.9213649&searchKeyword='
driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

In [12]:
# 검색창에 서울 검색
# 검색창 ; textarea
search_box = driver.find_element(by=By.NAME, value='searchKeyword')  # BY.대문자
search_box.send_keys('서울')
search_box.send_keys(Keys.ENTER)
time.sleep(2)

In [13]:
# page
p = [ str(i+1) for i in range(10) ]
line = []


for j in range(len(p)):  # page loop

    # station 정보 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    stn = soup.select('div .area-location-list > ul > li')
    stn

    #for i in [0]:
    for i in range(len(stn)):
        #print(stn[i])
        # 관서명
        name = stn[i].select_one('span.title').text.strip()
        print(name)
        # 주소
        addr = stn[i].select_one('address').text.strip()
        print(addr)
        # 전화번호
        phne = stn[i].select_one('span.tel').text.strip()
        print(phne)

        line.append([name, addr, phne])

    if int(p[j])%2 == 0:
        next_p_btn2 = driver.find_element(by=By.CSS_SELECTOR, value='.next_page')  # BY.대문자
        next_p_btn2.click()
        time.sleep(4)
    else:
        # 다음페이지
        next_p_btn1 = driver.find_element(by=By.XPATH, value='//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a')  # BY.대문자
        next_p_btn1.click()
        time.sleep(4)

동작소방서
서울특별시 동작구 여의대방로16길 55(신대방동)
02-847-1190
서대문소방서
서울특별시 서대문구 연희로 182(연희동)
02-3144-1190
광진소방서
서울특별시 광진구 광나루로 480(구의동)
02-457-0119
송파소방서
서울특별시 송파구 오금로51길 56(마천동)
02-403-2119
양천소방서
서울특별시 양천구 목동서로 180(목동)
02-2655-1119
은평소방서
서울특별시 은평구 통일로 962(진관동)
02-355-0119
노원소방서
서울특별시 노원구 한글비석로 1길 8(하계동)
02-977-0119
종로소방서
서울특별시 종로구 종로1길 28(수송동)
02-735-6119
도봉소방서
서울특별시 도봉구 도봉로 666(방학동)
02-3492-3437
용산소방서
서울특별시 용산구 한강대로 167(한강로2가)
02-794-0119
마포소방서
서울특별시 마포구 창전로 76(신수동)
02-716-0604
강서소방서
서울특별시 강서구 양천로 550(등촌동)
02-3663-0119
구로소방서
서울특별시 구로구 경인로 408(고척동)
02-2618-0119
성동소방서
서울특별시 성동구 살곶이길 331(행당동)
02-2622-1777
관악소방서
서울특별시 관악구 관악로 97(봉천동)
02-874-8119
서초소방서
서울특별시 서초구 사평대로 67(반포동)
02-537-0119
중부소방서
서울특별시 중구 퇴계로 394(무학동)
02-2234-0119
강동소방서
서울특별시 강동구 성내로 39(성내동)
02-470-0119
강북소방서
서울특별시 강북구 한천로 911
02-6946-0119
성북소방서
서울특별시 성북구 종암로 27길 3(종암동)
02-921-0119
동대문소방서
서울특별시 동대문구 장한로 34(장안동)
02-2249-0119
영등포소방서
서울특별시 영등포구 문래로 197(영등포동4가)
02-2675-0119
중랑소방서
서울특별시 중랑구 신내로 183(신내동)
02-3423-0119
강남소방서
서울특별시 강남구 테헤란로 629(삼성동)


In [14]:
df = pd.DataFrame(line, columns=['관서명', '주소', '번호'])
df

,관서명,주소,번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119
...,...,...,...
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119


In [15]:
df.to_csv('../daou/서울소방청out')